# 🤖 Model Training & Validation

> Competition: {{ COMPETITION_NAME }}

---

## 📦 Setup

In [ ]:
import sys
sys.path.append('../../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from shared.utils import set_seed
from shared.models import LGBMWrapper, XGBWrapper, cross_validate
from shared.evaluation import classification_report, regression_report, print_metrics, oof_score
from shared.visualization import plot_feature_importance

set_seed(42)

%matplotlib inline
%load_ext autoreload
%autoreload 2

## ⚙️ Configuration

In [ ]:
PROCESSED_PATH = 'data/processed'
TARGET_COL = 'target'  # Update this
ID_COL = 'id'  # Update this
TASK = 'classification'  # or 'regression'

## 📂 Load Processed Data

In [ ]:
train = pd.read_csv(f'{PROCESSED_PATH}/train_processed.csv')
test = pd.read_csv(f'{PROCESSED_PATH}/test_processed.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

In [ ]:
# Prepare features
EXCLUDE_COLS = [TARGET_COL, 'fold', ID_COL]
FEATURE_COLS = [c for c in train.columns if c not in EXCLUDE_COLS]

X = train[FEATURE_COLS]
y = train[TARGET_COL]
folds = train['fold']
X_test = test[FEATURE_COLS]

print(f"Features: {len(FEATURE_COLS)}")

## 🚀 Model Training

### LightGBM

In [ ]:
lgb_params = {
    'objective': 'binary' if TASK == 'classification' else 'regression',
    'metric': 'auc' if TASK == 'classification' else 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'seed': 42
}

oof_lgb, lgb_models = cross_validate(
    LGBMWrapper,
    X, y, folds,
    params=lgb_params,
    task=TASK,
    num_boost_round=1000,
    early_stopping_rounds=50
)

In [ ]:
# Evaluate
if TASK == 'classification':
    metrics = classification_report(y, (oof_lgb > 0.5).astype(int), oof_lgb)
else:
    metrics = regression_report(y, oof_lgb)

print_metrics(metrics, 'LightGBM OOF Results')

In [ ]:
# Feature importance
importance_df = lgb_models[0].get_feature_importance()
plot_feature_importance(importance_df, n_features=20)

### XGBoost

In [ ]:
xgb_params = {
    'objective': 'binary:logistic' if TASK == 'classification' else 'reg:squarederror',
    'eval_metric': 'auc' if TASK == 'classification' else 'rmse',
    'max_depth': 6,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

oof_xgb, xgb_models = cross_validate(
    XGBWrapper,
    X, y, folds,
    params=xgb_params,
    task=TASK,
    num_boost_round=1000,
    early_stopping_rounds=50
)

In [ ]:
if TASK == 'classification':
    metrics = classification_report(y, (oof_xgb > 0.5).astype(int), oof_xgb)
else:
    metrics = regression_report(y, oof_xgb)

print_metrics(metrics, 'XGBoost OOF Results')

## 🎯 Ensemble

In [ ]:
# Simple averaging
oof_ensemble = (oof_lgb + oof_xgb) / 2

if TASK == 'classification':
    metrics = classification_report(y, (oof_ensemble > 0.5).astype(int), oof_ensemble)
else:
    metrics = regression_report(y, oof_ensemble)

print_metrics(metrics, 'Ensemble OOF Results')

## 💾 Save Models & OOF Predictions

In [ ]:
import os
os.makedirs('../models', exist_ok=True)

# Save models
for i, model in enumerate(lgb_models):
    model.save(f'../models/lgb_fold{i}.pkl')

for i, model in enumerate(xgb_models):
    model.save(f'../models/xgb_fold{i}.pkl')

In [ ]:
# Save OOF predictions
oof_df = pd.DataFrame({
    ID_COL: train[ID_COL] if ID_COL in train.columns else range(len(train)),
    'oof_lgb': oof_lgb,
    'oof_xgb': oof_xgb,
    'oof_ensemble': oof_ensemble,
    'target': y
})
oof_df.to_csv(f'{PROCESSED_PATH}/oof_predictions.csv', index=False)
print("💾 OOF predictions saved!")

---
**Next Steps**: Proceed to `04_submission.ipynb`